In [1]:
import torch, torchvision
import sys # Python system library needed to load custom functions
import math # module with access to mathematical functions
import os # for changing the directory

import numpy as np  # for performing calculations on numerical arrays
import pandas as pd  # home of the DataFrame construct, _the_ most important object for Data Science
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt  # allows creation of insightful plots

sys.path.append('../../audio_preprocessing')
sys.path.append('../../src')
sys.path.append('../../model_training_utils')


import preprocessing_func_3
from generator_to_dataset_3 import NormalisedDataSet
from gdsc_utils import PROJECT_DIR
import model_training
import model_eval

os.chdir(PROJECT_DIR) # changing our directory to root

In [2]:
df_big_data = pd.read_csv('data/big_processed_data.csv')
df_big_argumented_data = pd.read_csv('data/big_argumentation_data.csv')
df = pd.concat([df_big_data, df_big_argumented_data], ignore_index=True)
df.tail()

,Unnamed: 0,file_path,label
66835,44959,data/big_data_upsample/44959.wav,65
66836,44960,data/big_data_upsample/44960.wav,65
66837,44961,data/big_data_upsample/44961.wav,65
66838,44962,data/big_data_upsample/44962.wav,65
66839,44963,data/big_data_upsample/44963.wav,65


In [3]:
import json

with open('audio_preprocessing/saved_data/upsampled_data_size_128_512.json') as f:
    my_info = json.load(f)

mean, std, class_weights = my_info["mean"], my_info["std"], my_info["weights"]

In [4]:
train_df_list = []
val_df_list = []

for i in range(66):
    my_df = df[df["label"] == i]
    current_train_df, current_val_df = train_test_split(my_df, test_size=0.2)
    train_df_list.append(current_train_df)
    val_df_list.append(current_val_df)

df_train = pd.concat(train_df_list, ignore_index=True)
df_val = pd.concat(val_df_list, ignore_index=True)

In [5]:
df_train.shape, df_val.shape

((53441, 3), (13399, 3))

In [6]:
train_dataset = NormalisedDataSet(
    non_normalised_data_generator_fn=preprocessing_func_3.non_normalised_data_generator, 
    normalised_data_generator_fn=preprocessing_func_3.normalised_data_generator,
    df=df_train, 
    mean=mean,
    std=std,
    shuffle=True
)

val_dataset = NormalisedDataSet(
    non_normalised_data_generator_fn=preprocessing_func_3.non_normalised_data_generator, 
    normalised_data_generator_fn=preprocessing_func_3.normalised_data_generator,
    df=df_val, 
    mean=mean,
    std=std,
    shuffle=False
)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=28)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=28)

In [7]:
device = model_training.get_device()

In [11]:
from torchvision.models import regnet_y_3_2gf
import torch.optim as optim
import torch.nn as nn

#resnet_model = resnet34(weights=ResNet34_Weights.DEFAULT)
regnet_model = regnet_y_3_2gf(num_classes=66)
regnet_model.stem[0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
regnet_model = regnet_model.to(device)

In [12]:
print(regnet_model)

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): Conv2dNormActivation(
          (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): Conv2dNormActivation(
            (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): Conv2dNormActivation(
            (0): Conv2d(72, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
            

In [13]:
optimizer = optim.Adam(regnet_model.parameters(), amsgrad=True)
loss = nn.CrossEntropyLoss()

In [14]:
model_training.training(
    model=regnet_model, 
    optimizer=optimizer, 
    loss_fn=loss, 
    train_dataloader=train_dataloader, 
    val_dataloader=val_dataloader, 
    model_path="models/regnet_y_3_2gf", 
    start_epoch=0,
    epochs=500,
    early_stop_thresh=50,
)

End of epoch 0: training accuracy = 72.51%, training loss = 0.950564433975197, training time taken = 864.11 seconds
End of epoch 0: validation accuracy = 91.42%, validation loss = 0.27350407967870166, validation time taken = 190.31 seconds
End of epoch 1: training accuracy = 93.71%, training loss = 0.20513318445764758, training time taken = 676.35 seconds
End of epoch 1: validation accuracy = 96.14%, validation loss = 0.12993240548248822, validation time taken = 148.21 seconds
End of epoch 2: training accuracy = 96.71%, training loss = 0.10476102307737664, training time taken = 682.23 seconds
End of epoch 2: validation accuracy = 97.63%, validation loss = 0.07016854326849552, validation time taken = 148.61 seconds
End of epoch 3: training accuracy = 98.03%, training loss = 0.06612266325488028, training time taken = 671.39 seconds
End of epoch 3: validation accuracy = 97.61%, validation loss = 0.07594779928864595, validation time taken = 144.89 seconds
End of epoch 4: training accuracy 

KeyboardInterrupt: 

In [15]:
torch.save(regnet_model, 'models/regnet_y_3_2gf/regnet_model_epoch_9.pth')